In [ ]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- setup ---
pd.set_option('max_columns', 50)

In [ ]:
vaccination = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
vaccination

In [ ]:
europe_country_list =list([
    'Austria','Belgium','Bulgaria','Czechia','Denmark','France','Germany','Greece',
    'Italy','Luxembourg','Lithuania','Norway','Poland','Portugal','Romania','Slovakia','Slovenia',
    'Spain', 'United Kingdom', 'Russia', 'Serbia', 'Ukraine'])

vaccination = vaccination[vaccination['country'].isin(europe_country_list)]
vaccination.shape

In [ ]:
vaccination = vaccination.drop(['iso_code', 'source_name', 'source_website'], axis = 1).reset_index()
vaccination

In [ ]:
vcc_all = vaccination.drop(['vaccines'], axis = 1)
vcc_all = vcc_all.groupby(['date']).sum()
vcc_all.reset_index(inplace=True)
vcc_all

In [ ]:

vaccination.fillna(method='ffill', inplace=True)
countries = vaccination.groupby('country')['total_vaccinations'].max().sort_values(ascending=False)[:10].index.tolist()

line_plots = []
for c in countries:
    vacc_data = vaccination[vaccination.country == c]
    line_plots.append(
        go.Scatter(
            name = c,
            x = vacc_data.date,
            mode='lines',
            y=vacc_data['total_vaccinations'],
        )
    )
    
fig = go.Figure(line_plots)
fig.update_layout(
    title ="Rate of vaccinations for the top-10 vaccinated countries",
    yaxis_title="Count",
    hovermode='x',
    legend_orientation = 'h',

)


fig.show()

In [ ]:

fig = px.line(vaccination, x="date", y="people_vaccinated_per_hundred", color='country', 
              title="date vs people_vaccinated_per_hundred")
fig.show()

In [ ]:
# Grouping data by vaccines
vaccines = vaccination.groupby(['country', 'vaccines']).count().reset_index()[['country', 'vaccines']]\
                  .groupby('vaccines').count()['country'].reset_index().sort_values('country', ascending=False)

# Select only the required columns
vaccines.columns = ['Vaccines', 'Number of countries used']

fig = px.bar(vaccines, x='Vaccines', y= 'Number of countries used')
fig.show()